In [245]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
import functions as fn
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import importlib
from fpdf import FPDF

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [246]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def print_values(self):
        print("\n")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

# Please fill in all inputs and their units below
# v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v 

# Constants
<br>For Reference: Units Set: psi, in

In [247]:
YOUNGSMODULUS = 30 * (10**6)
YM_UNIT = "psi"

MOI = 5 * (10**(-6))
MOI_UNIT = "m4"

POISSON_RATIO = 0.3

THICKNESS = 1

#options are "Circular" or "Square"
cross_section = "Circular"

W2 = 5000
W3 = 5000 
dist_load_length = 4 #length that the distributed load acts upon in the corresponding units to W3
load_direction = -1 #negative 1 if distributed load points down, +1 otherwise
loaded_elements = [2]
FORCE_UNIT = "N"

element_node_connections = np.array([[1,2,3], [1,3,4]])
print(f"number of elements: {len(element_node_connections)}")

coords = np.array([[0,0], [20,0], [20, 10], [0, 10]])

LENGTHS_UNIT = "m"
LENGTHS_CONVERSION_FACTOR = 1 #input conversion factor if inputted coordinate values are not in final units
#1 if no conversion needed

problem_number = 6

homework_number = 67




number of elements: 2


# Boundary Conditions
#### for displacements: [node#, u, v] -> assumed to be unknown (None) -> only input if known
#### for forces: [node#, Fx, Fy] -> assumed to be 0 -> only input if not 0

<br>Please note, node# are inputted straight from the diagram - do not adjust for python counting

In [248]:
disp_bcs = np.array([[1, 0, 0], [4, 0, 0]])

force_bcs = np.array([[1, None, None], [4, None, None]])

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Create our final output pdf

In [249]:
filename = f'HW{homework_number} Problem {problem_number} Numerical Outputs.txt'
f = open(filename, 'w')

f = open(filename, 'a')
f.write(f"Numerical Outputs for HW{str(homework_number)} Problem {str(problem_number)}\n\n")

38

In [250]:
E = Input("Young's Modulus", YOUNGSMODULUS, YM_UNIT)
E.print_values()

I_ = Input("Moment of Inertia", MOI, MOI_UNIT)
I_.print_values()

nodes = Input("Node numbers", element_node_connections, "None")
nodes.print_values()

coords *= LENGTHS_CONVERSION_FACTOR

node_coords = Input("Node Coordinates (x, y)", coords, "None")
node_coords.print_values()

total_length = np.max(node_coords.value)
print(f"\ntotal length: {total_length} {LENGTHS_UNIT}")



name: Young's Modulus
value: 30000000
unit: psi


name: Moment of Inertia
value: 4.9999999999999996e-06
unit: m4


name: Node numbers
value: [[1 2 3]
 [1 3 4]]
unit: None


name: Node Coordinates (x, y)
value: [[ 0  0]
 [20  0]
 [20 10]
 [ 0 10]]
unit: None

total length: 20 m


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [251]:
count_elems = len(nodes.value)
print(f"number of elements: {count_elems}")

count_nodes = nodes.value.max()
print(f"number of nodes: {count_nodes}")

dimension = (2*count_nodes)
print(f"dimension = {dimension}")

number of elements: 2
number of nodes: 4
dimension = 8


In [266]:
D_ = fn.create_D(POISSON_RATIO, YOUNGSMODULUS)
print(f"[D] = \n{D_}")

global_K = np.zeros((dimension, dimension))
print(f"Global K array dimensions: {global_K.shape}")

loose_ft = np.array([]) 

for i in range(count_elems):
    i_, j_, m_ = nodes.value[i]
    i_ -= 1; j_ -= 1; m_ -= 1
    print(f"i = {i_}, j = {j_}, m = {m_} for element {i+1}")

    print(coords[i_], coords[j_], coords[m_])

    area = fn.triangle_area(coords[i_], coords[j_], coords[m_])
    print(area)

    alpha, beta, gamma, abg_stacked = fn.alpha_beta_gamma(coords[i_], coords[j_], coords[m_])
    print(f"beta = {beta}\ngamma = {gamma}\nalpha = {alpha}\nabg stacked = \n{abg_stacked}")

    B_ = fn.create_B(beta, gamma, area)
    #print(f"[B] = \n{B_}")

    K_ = THICKNESS * area * ((np.transpose(B_)@D_)@B_)
    #print(f"[K] = \n{K_}\nK_ shape: {np.shape(K_)}")

    adjusted_K = fn.adjust_K_CST(K_, i_, j_, m_, count_nodes)
    print(f"adjusted K for element {1+i} = \n{adjusted_K}")
    
    global_K = np.add(global_K, adjusted_K)

    if (i+1) in loaded_elements:
        integrated_N_t = fn.integrate_N_t(alpha, beta, gamma, 20, 10, 1, [0,-5], 100)
        print(f"integrated Nt for element {1+i} = \n{integrated_N_t}")

        loose_ft = np.append(loose_ft, )

        

    

print(f"\nNew Global K = \n{global_K}")

[D] = 
[[32967032.96703297  9890109.89010989        0.        ]
 [ 9890109.89010989 32967032.96703297        0.        ]
 [       0.                0.         11538461.53846154]]
Global K array dimensions: (8, 8)
i = 0, j = 1, m = 2 for element 1
[0 0] [20  0] [20 10]
100.0
beta = [-10  10   0]
gamma = [  0 -20  20]
alpha = [200   0   0]
abg stacked = 
[[200   0   0]
 [-10  10   0]
 [  0 -20  20]]
missings = [3]
missing = 3
Case 2
adjusted K for element 1 = 
[[  8241758.24175824         0.          -8241758.24175824
    4945054.94505494         0.          -4945054.94505494
          0.                 0.        ]
 [        0.           2884615.38461538   5769230.76923077
   -2884615.38461538  -5769230.76923077         0.
          0.                 0.        ]
 [ -8241758.24175824   5769230.76923077  19780219.78021978
  -10714285.71428571 -11538461.53846154   4945054.94505494
          0.                 0.        ]
 [  4945054.94505494  -2884615.38461538 -10714285.71428571
   358516

TypeError: _append_dispatcher() missing 2 required positional arguments: 'arr' and 'values'

In [262]:

integrated_N_t = fn.integrate_N_t(alpha, beta, gamma, 20, 10, 1, [0,-5], 100)
print(integrated_N_t)

adjusted_N_t = fn.adjust_K_CST(integrated_N_t, i_, j_, m_, count_nodes)

x = 20, y = 10, z = 1
i= 0
i= 1
i= 2
i= 3
i= 4
i= 5
[  0.  -0.   0. -50.   0. -50.]


In [128]:
LENGTHS = np.zeros(count_elems)
print(LENGTHS)

for i in range(count_elems):

    connection = nodes.value[i]
    N1 = connection[0]
    N2 = connection[1]

    x1 = (node_coords.value[N1-1])[0]
    y1 = (node_coords.value[N1-1])[1]

    x2 = (node_coords.value[N2-1])[0]
    y2 = (node_coords.value[N2-1])[1]

    LENGTHS[i] = f.get_element_length(x1, y1, x2, y2)

    print(f"element {i+1}: length = {LENGTHS[i]}")

print(LENGTHS)

lengths = Input("Lengths", LENGTHS, LENGTHS_UNIT)
lengths.print_values()


[0. 0.]


AttributeError: '_io.TextIOWrapper' object has no attribute 'get_element_length'

two reactions per node
<br>v_phi_list -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.


In [ ]:
v_phi_list = [None]*(dimension - len(HINGE_NODES))
print("displacement bcs: [node number, y disp (v), angle of rotation (phi)]")
print(disp_bcs)

f.adjust_array(v_phi_list, disp_bcs, "displacements")
for node in HINGE_NODES:
    v_phi_list.insert((node*2)+2, None)
print("v & phi list:")
print(v_phi_list)

Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [ ]:
force_list = [0]*(dimension - len(HINGE_NODES))
#force_list = np.zeros(((count_nodes*2), 1))

nodesWithLoads = []

loadedNodes = 0

for i in range(len(force_bcs)):
    if force_bcs[i,1] != None or force_bcs[i,2] != None:
        loadedNodes += 1
        nodesWithLoads.append((force_bcs[i,0] - 1))

for i in range(len(force_list)):
    if v_phi_list[i] == 0:
        force_list[i] = None
    elif v_phi_list[i] is None:
        force_list[i] = 0

#adjust force array given concentrated loading conditions
adjust_array(force_list, force_bcs, "forces")

#add in extra reactions for hinges
for node in HINGE_NODES:
    force_list.insert((node*2)+2, 0)

print(f"number of nodes with concentrated loads: {loadedNodes}")
print(f"at nodes: {nodesWithLoads}")

print(f"force list: \n{force_list}")

for i in range(len(force_list)):
    if force_list[i] == v_phi_list[i]:
        print(f"Error in checking BCs: index {i} in forcelist is equal to displist")

### Accounting for surface traction

In [ ]:
if W3 != 0:
    m_constant = (W3 - W2)/dist_load_length

    def line(x, b):
        return (m_constant * x) + b if W3 != W2 \
            else W3    
    
    loose_ft = [0] * (count_elems * 4)

    fw = [0] * (dimension - len(HINGE_NODES))
    #for each element, create local fw and transport to global
    for i in range(count_elems):
        if (i+1) in loaded_elements:
            b_constant = sum_until_n(lengths.value, (i-1)) * m_constant
            print(f"for element {i+1}: b constant: {b_constant}")

            tStar1 = lambda x: (create_NT(x, lengths.value[i]))[0,0] * load_direction * line(x, b_constant)
            tStar2 = lambda x: (create_NT(x, lengths.value[i]))[0,1] * load_direction * line(x, b_constant)
            tStar3 = lambda x: (create_NT(x, lengths.value[i]))[0,2] * load_direction * line(x, b_constant)
            tStar4 = lambda x: (create_NT(x, lengths.value[i]))[0,3] * load_direction * line(x, b_constant) 

            ft1, error = integrate.quad(tStar1, 0, lengths.value[i])
            ft2, error = integrate.quad(tStar2, 0, lengths.value[i])
            ft3, error = integrate.quad(tStar3, 0, lengths.value[i])
            ft4, error = integrate.quad(tStar4, 0, lengths.value[i]) 

            fw[2*i] += ft1
            fw[(2*i) + 1] += ft2
            fw[(2*i) + 2] += ft3
            fw[(2*i) + 3] += ft4

            loose_ft[(4*i)] = ft1
            loose_ft[(4*i) + 1] = ft2
            loose_ft[(4*i) + 2] = ft3
            loose_ft[(4*i) + 3] = ft4

            print(f"ft1 = {ft1}, ft2 = {ft2}, ft3 = {ft3}, ft4 = {ft4}")
    print(f"loose ft = {loose_ft}, length = {len(loose_ft)}")
else:
    fw = 0

for node in HINGE_NODES:
    if np.isscalar(fw) is False:
        fw.insert((node*2)+1, 0)
        print(f"inserted 0 into fw at index: {(node*2) + 1}")
        print(f"new length of fw: {len(fw)}")

print(f"fw = {fw}")


Global [k] matrix is a symmetric array of shape (dimension, dimension)
<br>beam constants are one per element, stored in an array of shape (number of elements, 1).

In [ ]:


beam_constant = np.zeros((count_elems,1))
for i in range(count_elems):
    print(f"E = {E.value}, I = {I_.value}, length for element {i+1} = {lengths.value[i]}")
    beam_constant[i] = (E.value*I_.value)/(lengths.value[i]**3)

print(f"beam constants = \n{beam_constant}")

## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [ ]:
hinge_counter = 0
for i in range(count_elems):
    local_K = create_beam_local_k(lengths.value[i])
    local_K *= beam_constant[i]
    print(f"\nk matrix for element {i+1}: \n{local_K}")
    #print(local_K)
    start_node = nodes.value[i,0]
    end_node = nodes.value[i,1]
    print(f"Element {i+1} starting at node: {start_node} and ending at node: {end_node}")
    #print(np.vsplit(local_K, 2))
    Sn = (start_node - 1)*2
    En = (end_node - 1)*2

    x = Sn + hinge_counter
    y = x

    if HINGE_NODES is not None and (start_node - 1) in HINGE_NODES:
        row_to_insert = np.zeros((1,4))
        col_to_insert = np.zeros((5))

        local_K = np.insert(local_K, 1, row_to_insert, axis = 0)
        local_K = np.insert(local_K, 1, col_to_insert, axis = 1)

        print(f"\nHinge modified K Matrix for element {i+1}: \n{local_K}")

        hinge_counter += 1
    print(f"\nk matrix for element {i+1} inputted at global matrix position {x},{y}")

    global_K[x:x+local_K.shape[0], y:y+local_K.shape[1]] += local_K

print(f"\nGlobal K matrix: \n{global_K}")
unmodified_global_K = global_K.copy()
check_symmetric(global_K)
print(f"Zero rows at: {np.where(~global_K.any(axis=1))[0]}")

In [ ]:
if W3 != 0:  
    for i in range(len(force_list)):
        if force_list[i] is not None:
            force_list[i] += fw[i]
        else: force_list[i] = fw[i]
print(force_list)

In [ ]:
j = 0
print(np.shape(global_K))
while j < dimension:
    print(f"{j}")

    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #global_K to reflect the changes.
    if v_phi_list[j] == 0:
        print(f"u = 0 at row: {j}")
        force_list[j] = 0 
        global_K[j] = 0
        global_K[j,j] = 1
        j+=1
    elif v_phi_list[j] is None:
        print(f"unknown u/phi at row {j}")
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print(f"row {j} altered")
        print(v_phi_list[j])
        force_list[j] = v_phi_list[j]
        for c in range(count_nodes):
            global_K[j,c] = 1 if global_K[j,c] > 0 else 0
        j+=1


print(f"global K -> \n{global_K}")
print(f"force list -> \n{force_list}")

In [ ]:
#global_K.round(0)
global_K_inv = inv(global_K)
#print(global_K_inv)

node = 1

final_u_phi = np.matmul(global_K_inv, force_list)

print(f"Final u and phi: {final_u_phi}")

rounded_u_phi = [round(u_phi , 6) for u_phi in final_u_phi]
f.write(f"\nDisplacements, Angles of Rotation = \n{rounded_u_phi}" 
        f"\n{LENGTHS_UNIT}, rad")

### Finding forces

In [ ]:
elem_forces_moments = [0] * (count_elems*4)
force_hinge_counter = 0
try:
    loose_ft
    print(f"loost ft length = {len(loose_ft)}")
except NameError:
    print("no loose_ft")
    
for i in range(count_elems):
    start_node = nodes.value[i,0]
    if W3 != 0:
        local_K = create_beam_local_k(lengths.value[i])
        local_K *= beam_constant[i]

        mover = 4*i
        print(f"i = {i}, mover = {mover}")

        indexer = 2*i + force_hinge_counter

        print(f"local K element {i+1} = {local_K}")
        if HINGE_NODES is not None and (start_node - 1) in HINGE_NODES:
            #if we have hinges and the element begins on a hinge
            print(f"Code FF01: at element {i+1}")
            local_U = np.array([
            [final_u_phi[indexer]],
            [final_u_phi[indexer + 2]], 
            [final_u_phi[indexer + 3]], 
            [final_u_phi[indexer + 4]]
            ])

            local_fw = np.array([
            [loose_ft[mover]], 
            [loose_ft[mover + 1]], 
            [loose_ft[mover + 2]], 
            [loose_ft[mover + 3]]
            ])

            force_hinge_counter += 1
            print(f"\nlocal fw for element {i+1} = {local_fw}")

        else:
            print(f"Code FF02: at element {i+1}")
            local_U = np.array([
            [final_u_phi[2*i + force_hinge_counter]],
            [final_u_phi[(2*i) + 1 + force_hinge_counter]], 
            [final_u_phi[(2*i) + 2 + force_hinge_counter]], 
            [final_u_phi[(2*i) + 3 + force_hinge_counter]]
            ])

            local_fw = np.array([
            [loose_ft[mover]], 
            [loose_ft[mover + 1]], 
            [loose_ft[mover + 2]], 
            [loose_ft[mover + 3]]
            ])

            print(f"\nLocal U phi for element {i+1} : \n{local_U}")
        local_forces = np.subtract((local_K @ local_U), local_fw)

    else:
        local_K = create_beam_local_k(lengths.value[i])
        local_K *= beam_constant[i]

        indexer = 2*i + force_hinge_counter
        
        if HINGE_NODES is not None and (start_node - 1) in HINGE_NODES:
            print(f"Code FF03: at element {i+1}")
            local_U = np.array([
            [final_u_phi[indexer]],
            [final_u_phi[indexer + 2]], 
            [final_u_phi[indexer + 3]], 
            [final_u_phi[indexer + 4]]
            ])

            force_hinge_counter += 1
        
        else:
            print(f"Code FF04: at element {i+1}")
            local_U = np.array([
            [final_u_phi[indexer]],
            [final_u_phi[indexer + 1]], 
            [final_u_phi[indexer + 2]], 
            [final_u_phi[indexer + 3]]
            ])
        local_forces = local_K @ local_U

    #negatives convert to EB sign convention
    #local_forces[1,0] *= -1
    #local_forces[2,0] *= -1

    elem_forces_moments[4*i] = local_forces[0,0]
    elem_forces_moments[4*i + 1] = local_forces[1,0]
    elem_forces_moments[4*i + 2] = local_forces[2,0]
    elem_forces_moments[4*i + 3] = local_forces[3,0]

    print(f"local forces (element {i+1}) [{FORCE_UNIT}] = \n{local_forces}")

print(f"collected element forces [{FORCE_UNIT}] = \n{elem_forces_moments}")
rounded_f_m = [round(element , 4) for element in elem_forces_moments]
f.write(f"\n\nForces, Moments = \n{rounded_f_m}\n{FORCE_UNIT}, {FORCE_UNIT}*{LENGTHS_UNIT}")


## Finding Stresses

In [ ]:
elem_forces = [0] * (count_elems * 2)
elem_moments = [0] * (count_elems * 2)

for i in range(len(elem_forces_moments)):
    if check_even(i) == True:
        elem_forces[int(i / 2)] = elem_forces_moments[i]
    else:
        elem_moments[int((i-1) / 2)] = elem_forces_moments[i]

print(f"elem forces (fea sign convention) = \n{elem_forces}")
print(f"elem moments (fea sign convention) = \n{elem_moments}")

abs_forces = [abs(element) for element in elem_forces]
abs_moments = [abs(element) for element in elem_moments]

max_force = max(abs_forces)
max_moment = max(abs_moments)

max_moment_AB = max((abs_moments[:8]))

print(f"max force = {max_force}, max moment = {max_moment}")

if cross_section == "Circular":
    print("SM Code 1: circular cross section")
    radius = ((4*I_.value)/m.pi)**0.25
    max_trans_shear = (4*max_force)/(3*((m.pi)*(radius**2)))
    max_bending_stress = (max_moment * radius)/(I_.value)

    max_bending_stress_seg_AB = (max_moment_AB * radius)/(I_.value)

elif cross_section == "Square":
    print("SM Code 2: square cross section")
    side_length = (12 * I_.value)**0.25
    max_trans_shear = (3*max_force)/(2*(side_length**2))
    max_bending_stress = (max_moment * side_length * 0.5)/(I_.value)

else:
    print(f"Error SM1: Calculating for a {cross_section} cross section is not supported.")

print(f"\nmaximum transverse shear stress = {max_trans_shear} {YM_UNIT}")
f.write(f"\n\nMaximum transverse shear stress = {str(round(max_trans_shear, 4))} {YM_UNIT}")

print(f"\nmaximum bending stress = {max_bending_stress} {YM_UNIT}")
f.write(f"\n\nMaximum bending stress = {str(round(max_bending_stress, 4))} {YM_UNIT}")
try:
    max_bending_stress_seg_AB
    f.write(f"\n\nMaximum bending stress (segment AB) = {str(round(max_bending_stress_seg_AB, 4))} {YM_UNIT}")
except NameError:
    print("no special segments")

In [ ]:
print(max(abs_moments[:8]))

In [ ]:
#Shear Force Diagram
for i in range(len(elem_forces)):
    if check_even(i) == False:
        elem_forces[i] *= -1

shear_y_values = elem_forces
shear_y_values = [round(elem, 4) for elem in shear_y_values]
shear_y_values.append(0)
print(f"y values = {shear_y_values}")

x_values = [0] * len(elem_forces)

for i in range(count_elems):
    print(i)
    x_values[2*i] += node_coords.value[i][0]
    x_values[(2*i) + 1] += node_coords.value[i+1][0]

x_values.append(x_values[len(elem_forces) - 1])

print(f"x values = {x_values}")

shear_fig = plt.figure("Shear Figure")

figure, ax = plt.subplots(nrows=3,ncols=1,sharex=True,figsize=(8,14))
#plt.rcParams["figure.figsize"] = [10, 14]
#plt.rcParams["figure.autolayout"] = True

ax[0].set_title(f"HW{homework_number} Problem {problem_number} FEA Shear Force Diagram")
ax[0].set_xlim(0, total_length * 1.1)
ax[0].axhline(0, color="black", linestyle = "--")
ax[0].axvline(0, color="black", linestyle = "--")
ax[0].set_ylabel(f"V [{FORCE_UNIT}]")
ax[0].set_xlabel(f"x [{LENGTHS_UNIT}]")
#plt.subplot(3,1,1)
ax[0].plot(x_values, shear_y_values, color = "c")

#shear_fig.savefig(f"HW{homework_number} Problem {problem_number} Shear Figure")

#Bending Moment Diagram
#convert element moments into E-B sign convention
for i in range(len(elem_moments)):
    if check_even(i) == True:
        elem_moments[i] *= -1

moment_y_values = elem_moments
moment_y_values = [round(elem, 4) for elem in moment_y_values]
moment_y_values.append(0)
print(f"y values = {moment_y_values}")

print(f"x values = {x_values}")

moment_fig = plt.figure("Moment Figure")

ax[1].set_title(f"HW{homework_number} Problem {problem_number} FEA Bending Moment Diagram")
ax[1].set_xlim(0, total_length * 1.1)
ax[1].axhline(0, color="black", linestyle = "--")
ax[1].axvline(0, color="black", linestyle = "--")
ax[1].set_ylabel(f"M [{FORCE_UNIT}*{LENGTHS_UNIT}]")
ax[1].set_xlabel(f"x [{LENGTHS_UNIT}]")
ax[1].plot(x_values, moment_y_values, color = "m")

#mom.savefig(f"HW{homework_number} Problem {problem_number} Moment Figure")

#Displacement Figure
displacement_fig = plt.figure("Displacement Field")

disp_hinge_counter = 0

for i in range(count_elems):
    start_node = nodes.value[i,0]
    print(f"local x adjustment factor = {node_coords.value[i][0]}")
    print(f"element {i+1}: start node {start_node}")

    disp_x_values = np.linspace(node_coords.value[i][0], node_coords.value[i+1][0], 200)
    print(disp_x_values[0], disp_x_values[199])
    
    if HINGE_NODES is not None and (start_node - 1) in HINGE_NODES:
        print(f"Element {i+1}: DF Code 1")
        u1 = final_u_phi[2*i + disp_hinge_counter]
        u2 = final_u_phi[(2*i) + 2 + disp_hinge_counter]
        u3 = final_u_phi[(2*i) + 3 + disp_hinge_counter]
        u4 = final_u_phi[(2*i) + 4 + disp_hinge_counter]
        print([u1, u2, u3, u4])

        disp_hinge_counter += 1

    else:  
        print(f"Element {i+1}: DF Code 2")
        u1 = final_u_phi[2*i + disp_hinge_counter]
        u2 = final_u_phi[(2*i) + 1 + disp_hinge_counter]
        u3 = final_u_phi[(2*i) + 2 + disp_hinge_counter]
        u4 = final_u_phi[(2*i) + 3 + disp_hinge_counter]
        print([u1, u2, u3, u4])
        
    def y(x): 
            return (
                create_NT(x - node_coords.value[i][0], lengths.value[i])[0,0] * u1
                + create_NT(x - node_coords.value[i][0], lengths.value[i])[0,1] * u2
                + create_NT(x - node_coords.value[i][0], lengths.value[i])[0,2] * u3
                + create_NT(x - node_coords.value[i][0], lengths.value[i])[0,3] * u4
            ) 
    
    disp_y_values = list(map(y, disp_x_values))
    
    #for i in range(0, 200, 10):
     #   print(f"{disp_x_values[i]}, {disp_y_values[i]}")

    #plot labels
    ax[2].set_title(f"HW{homework_number} Problem {problem_number} FEA Displacement Field")
    ax[2].set_xlim(0, total_length * 1.1)
    ax[2].axhline(0, color="black", linestyle = "--")
    ax[2].axvline(0, color="black", linestyle = "--")
    ax[2].set_ylabel(f"v [{LENGTHS_UNIT}]")
    ax[2].set_xlabel(f"x [{LENGTHS_UNIT}]")

    ax[2].plot(disp_x_values, disp_y_values)

figure.savefig(f"HW{homework_number} Problem {problem_number} Collected Figures")





In [ ]:
f.close()